# Sentiment Analysis for Twitter

## Overview

This tutorial is going to introduce some simple tools for detecting sentiment in Tweets. We will be using a set of tools called the Natural Language Toolkit ([NLTK](http://www.nltk.org)). This is collection of software written in the [Python](https://www.python.org) programming language. An important design goal behind Python is that it should be easy to read and fun to use, so well-suited for beginners. A similar motivation inspired NLTK: it should make complex tasks easy to carry out, and it should be written in a way that would allow users to inspect and understand the code.

Why is this relevant? Well, a lot of software these days is built to be easy to use, but hard to inspect. For example, smartphones have a lot of slick apps on them, but very few people have the expertise to look under the hood to find out how they work. NLTK has quite the opposite approach: you are actively encouraged to discover how the code works. However, your level of understanding will depend heavily on how far you get to grips with Python itself.

This tutorial is written using the [Ipython](http://ipython.org) framework. This allows text to be interspersed by fragments of code, occuring in special "cells". Just below is a cell where we are using Python to do a simple calculation:

In [10]:
3 + 4

7

Some of the cells will contain snippets of code that are necessary for the big story to work, but which you don't need to understand. We'll try to make it clear when it's important for you to pay attention to one of the cells. 

## Twitter

As you know, people are tweeting all the time. The rate varies, with about 6,000 per second being the average, but when I last checked, the [rate was over 10,000 Tweets per second](http://www.internetlivestats.com/one-second/#tweets-band). So, a lot. Twitter kindly allows people to tap into a small sample of this stream &mdash; unless you're able to pay, the sample is at most 1% of the total stream. 

Here's a tiny snapshow to Tweets, taken at 13.00 pm on Sunday 11 November 2015. By using the keywords `'love, hate'`, we restrict our sample to just those Tweets containing one or both of those words.

In [2]:
import nltk # load up the NLTK library
from nltk.twitter import Twitter
tw = Twitter() # start a new client that connects to Twitter
tw.tweets(keywords='love, hate', limit=25) #filter Tweets from the public stream

We're flowing like a stream, running 
 Free, traveling on the wings of love 
#OTWOLWish"
RT @prttyjsmn: Oppa. Don't mind those haters of yours. They just insecure on what you have done. We love u #INSPIRITPHLovesLeaderGyu
I love this album its so incredible and you are such and inspiration question is would you date me if you was single ima girl btw #AskDemi x
Hi @Harry_Styles
I hope you're well.xx
So excited to hear Made In The AM,I love it already.
Mind following me sunshine?
-x5,182
RT @ddlovato: Learned to love myself a little sooner 💕🤗 #AskDemi https://t.co/hpG8cWNJ9X
RT @WeeklyChris: Its like if u have followers there's an expectation on content but I started this thing cause I just love making stuff &amp; t…
@ddlovato which song on the album Confident, you'd love to have a video clip? i love you so much, you're my life ❤ x2 #AskDemi
@ddlovato what would you say for people to believe in yourself? I love you more than everything #AskDemi ❤️ 1
RT @itsmeJoseManalo: YOU HATE ME?

GET

## Using a Twitter corpus

You too can sample Tweets in this way, but you'll need to set up your Twitter API keys according to [these instructions](http://www.nltk.org/howto/twitter.html). Since this is a bit of hassle, for the rest of this tutorial, we'll focus our attention on a sample of 20,000 Tweets that were collected at the end of April 2015. In order focus on Tweets about the UK general election, the public stream was filtered with the following set of terms:
```
david cameron, miliband, milliband, sturgeon, clegg, farage, tory, tories, ukip, snp, libdem
```
The following code cell allows us to get hold of this collection, and prints out the text of the first 15. You don't need to worry about the details of how this happens.

In [4]:
from nltk.corpus import twitter_samples
strings = twitter_samples.strings('tweets.20150430-223406.json')

In [5]:
for string in strings[:20]:
    print(string)

RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP
VIDEO: Sturgeon on post-election deals http://t.co/BTJwrpbmOY
RT @LabourEoin: The economy was growing 3 times faster on the day David Cameron became Prime Minister than it is today.. #BBCqt http://t.co…
RT @GregLauder: the UKIP east lothian candidate looks about 16 and still has an msn addy http://t.co/7eIU0c5Fm1
RT @thesundaypeople: UKIP's housing spokesman rakes in £800k in housing benefit from migrants.  http://t.co/GVwb9Rcb4w http://t.co/c1AZxcLh…
RT @Nigel_Farage: Make sure you tune in to #AskNigelFarage tonight on BBC 1 at 22:50! #UKIP http://t.co/ogHSc2Rsr2
RT @joannetallis: Ed Milliband is an embarrassment. Would you want him representing the UK?!  #bbcqt vote @Conservatives
RT @abstex: The FT is backing the Tories. On an unrelated note, here's a photo of FT leader writer Jonathan Ford (next to Boris) http://t.c…
RT @NivenJ1: “@George_Osborne: Ed Mi

## Sentiment Analysis

When we talk about understanding natural language, we often focus on 'who did what to whom'. Yet in many situations, we are more interested in attitudes and opinions. When someone writes about a movie, did they like it or hate it? Is a product review for a water bottle on Amazon positive or negative? Is a Tweet about the US President supportive
or critical? We might also care about the intensity of the views expressed: "this is a fine movie" is different from "WOW! This movie is soooooo great!!!!" even though both are positive.

*Sentiment analysis* (or *opinion mining*) is a broad term for a range of techniques that try to identify the subjective views expressed in texts. Many organisations care deeply about public opinion &mdash; whether these concern commercial products, creative works, or political parties and policies &mdash; and have consequently turned to sentiment analysis as a way of gleaning valuable insights from voluminous bodies of online text. This in turn has stimulated much activity in the area, ranging from academic research to commercial applications and industry-focussed conferences.

However, it's worth saying at the outset that sentiment analysis is hard. Although it is designed to work with written text, the way in which people express their feelings is often goes far beyond what they literally say. In spoken language, intonation will be important. And of course we often express emotion using no words at all, as illustrated in this picture from Darwin's book *The Expression of the Emotions*.

<a title="By Charles Darwin (author of volume); unknown photographer of plate [Public domain], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3APlate_depicting_emotions_of_grief_from_Charles_Darwin's_book_The_Expression_of_the_Emotions.jpg"><img align="center" width="512" alt="Plate depicting emotions of grief from Charles Darwin&'s book The Expression of the Emotions" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Plate_depicting_emotions_of_grief_from_Charles_Darwin%27s_book_The_Expression_of_the_Emotions.jpg/512px-Plate_depicting_emotions_of_grief_from_Charles_Darwin%27s_book_The_Expression_of_the_Emotions.jpg"/></a>

## Classifying sentences

Let's say that we want to classify a sentence into one of three categories: **positive**, **negative** or
**neutral**. Each of these can be illustrated by posts on Twitter collected during the UK General Election in 2015.

<dl>
<dt>positive:</dt>
<dd><i>Good stuff from Clegg. Clear, passionate & honest about the difficulties of govt but also the difference @LibDems have made.</i></dd>

<dt>negative:</dt> 
<dd><i>Hmm. Ed Miliband being against SNP is a bad move  I think. It'll cost him n it is a dumb choice.</i></dd>

<dt>neutral:</dt> 
<dd><i>Why is Ed Milliband trending when him name is Ed Miliband?</i></dd>
</dl>

The term **polarity** is often used to refer to whether a piece of text is judged to be positive or negative.

The easiest approach to classifying examples like these is to get hold of two lists of words, positive ones such as *good*, *excellent*, *fine*, *triumph*, *well*, *succeed*, ... and negative ones such as *bad*, *poor*, *dismal*,
*lying*, *fail*, *disaster*, .... We figure out an overall polarity score based on the ratio of positive tokens to negative ones in a given string. A sentence with neither positive or negative tokens (or possibly an equal number of each) will be categorised as neutral. This simple approach is likely to yield the roughly correct results for the Twitter examples above.

Things become more complicated when negation enter into the picture. The next example is mildly positive (at least in British English), so we need to ensure that *not* reverses the polarity of *bad* in appropriate contexts,

<i>Given Miliband personal ratings still 20 points behind Cameron, I'd say that not a bad margin for Labour leader  https://t.co/ILQP93VYLF</i>

## Classifying Tweets with VADER

[VADER](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) is a system for determining the sentiment of texts which has been incorporated into NLTK. It is based on the idea of looking for positive and negative words, but adds to important new elements. First, it uses a lexicon of 7,500 items which have been manually annotated for both polarity and intensity. Second, the overall score for an input text is computed by using a complex set of rules that take into account not just words (and negation), but also the boosting effect of devices like capitalisation and punctuation.

In [13]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [14]:
sia.polarity_scores("I am happy and virtuous")

{'compound': 0.7964, 'neg': 0.0, 'neu': 0.22, 'pos': 0.78}

In [6]:
full_tweets = twitter_samples.docs('tweets.20150430-223406.json')

In [39]:
import pandas as pd
from numpy import nan as NA
data = pd.DataFrame()
data['text'] = [t['text'] for t in full_tweets]

In [45]:
parties = {}
parties['conservative'] = set(['osborne', 'portillo', 'pickles', 'tory', 'tories',
                                'torie', 'voteconservative', 'conservative', 'conservatives', 'bullingdon', 'telegraph'])
parties['labour'] = set(['uklabour', 'scottishlabour', 'labour', 'lab', 'murphy'])
parties['libdem'] = set(['libdem', 'libdems', 'dems', 'alexander'])
parties['ukip'] = set(['ukip', 'davidcoburnukip'])
parties['snp'] = set(['salmond', 'snp', 'snpwin', 'votesnp', 'snpbecause', 'scotland',
                       'scotlands', 'scottish', 'indyref', 'independence', 'celebs4indy'])

leaders = {}
leaders['cameron'] = set(['cameron', 'david_cameron', 'davidcameron','dave', 'davecamm'])
leaders['miliband'] = set(['miliband', 'ed_miliband', 'edmiliband', 'edm', 'milliband', 'ed', 'edforchange', 'edforpm', 'milifandom'])
leaders['clegg'] = set(['clegg'])
leaders['farage'] = set(['farage', 'nigel_farage', 'nsegel', 'askfarage', 'asknigelfarage', 'asknigelfar'])
leaders['sturgeon'] = set(['sturgeon', 'nicola_sturgeon', 'nicolasturgeon', 'nicola'])

def tweet_classify(text, keywords, verbose=False):
    label = NA
    from nltk.tokenize import wordpunct_tokenize
    import operator
    toks = wordpunct_tokenize(text)
    toks_lower = [t.lower() for t in toks]
    d = {}
    for k in keywords:
        d[k] = len(keywords[k] & set(toks_lower))  
    
    best = max(d.items(), key=operator.itemgetter(1))
    if best[1] > 0:
        label = best[0]
    if verbose:
        print(text)
        print(best)
        print(label)
    return label


data['party'] = [tweet_classify(row['text'], parties) for index, row in data.iterrows()]
data['leader'] = [tweet_classify(row['text'], leaders) for index, row in data.iterrows()]

In [49]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
data['sentiment'] = [sia.polarity_scores(row['text'])['compound'] for index, row in data.iterrows()]

In [48]:
data.describe()

,sentiment
count,20000.000000
mean,0.060084
std,0.415121
min,-0.982100
25%,-0.205700
50%,0.000000
75%,0.401900
max,0.957100


In [52]:
data.sort()

,text,party,leader,sentiment
0,RT @KirkKus: Indirect cost of the UK being in ...,ukip,NaN,0.0000
1,VIDEO: Sturgeon on post-election deals http://...,NaN,sturgeon,0.0000
2,RT @LabourEoin: The economy was growing 3 time...,NaN,cameron,0.1779
3,RT @GregLauder: the UKIP east lothian candidat...,ukip,NaN,0.0000
4,RT @thesundaypeople: UKIP's housing spokesman ...,ukip,NaN,0.4588
5,RT @Nigel_Farage: Make sure you tune in to #As...,ukip,farage,0.3802
6,RT @joannetallis: Ed Milliband is an embarrass...,conservative,miliband,-0.4389
7,RT @abstex: The FT is backing the Tories. On a...,conservative,NaN,0.0258
8,RT @NivenJ1: “@George_Osborne: Ed Miliband pro...,NaN,miliband,0.0000
9,LOLZ to Trickle Down Wealth. It's never trickl...,NaN,NaN,0.8167


In [34]:
grouped_leader = data['sentiment'].groupby(data['leader'])
grouped_leader.mean()

leader
cameron     0.047176
clegg       0.103216
farage      0.088027
miliband    0.086034
sturgeon    0.041670
Name: sentiment, dtype: float64

In [30]:
grouped_party = data['sentiment'].groupby(data['party'])
grouped_party.mean()

party
conservative    0.098122
labour          0.001554
libdem          0.068198
snp             0.047162
ukip            0.103672
Name: sentiment, dtype: float64

In [35]:
grouped_leader.count()

leader
cameron     1472
clegg        688
farage      3079
miliband    7340
sturgeon     630
Name: sentiment, dtype: int64

In [36]:
grouped_party.count()

party
conservative    3130
labour          3403
libdem            58
snp             4100
ukip            2768
Name: sentiment, dtype: int64

In [32]:
grouped_party.min()

party
conservative   -0.9821
labour         -0.9231
libdem         -0.6705
snp            -0.9538
ukip           -0.9426
Name: sentiment, dtype: float64

In [ ]:
grouped_leader.count().sort_values(ascending=False)

## Challenges

It's not hard to find examples where something close to full natural language understanding is required to determine the correct polarity.

* <i>David Cameron doesn't seem to have done too badly until now. Otherwise #milifandom and #cleggers would be attacking him for these bad things.</i> 
* <i>Even though I don't like UKIP I'm hating them less and less every day, they do actually have very some good policies.</i> 

In [53]:
sia.polarity_scores("David Cameron doesn't seem to have done too badly until now. Otherwise #milifandom and #cleggers would be attacking him for these bad things.")

{'compound': -0.8625, 'neg': 0.324, 'neu': 0.676, 'pos': 0.0}

A further challenge in sentiment analysis is deciding the right level of **granularity** for the topic under discussion. Often, we can agree in the overall polarity of a sentence (or even of larger texts) because there is a single dominant topic. But in a list-like construction such as the following, different sentiments are associated with different
entities, and there is no sensible way of aggregating this into a combined polarity score for the text as a whole:

<i>@hugorifkind Audience - good. Mili - bad. Clegg - a bit sad. Cam - unscathed</i> 

Finally, our current approaches to language processing struggle with sarcasm, irony and satire, since these (intentionally) agin lead to polarity reversals.

* <i>LOVE being sat on a plane for 4 hours after a 10 hour flight !! Soooo fun !</i> 
* <i>The wrong spelling of Ed Miliband is trending, but not the correct one. Good job, Britain.</i> 